<a href="https://colab.research.google.com/github/Hamza-Ali0237/HF-Projects/blob/main/Beginner/Text%20Summarization%20App/Text_Summarization_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from transformers import pipeline

In [18]:
summarizer = pipeline(
    task = "summarization",
    model = "facebook/bart-large-cnn",
    device = 0
)

Device set to use cuda:0


In [19]:
# Sample text
text = """In the heart of a sprawling metropolis, where steel and glass pierce the sky,
          a vibrant energy pulses day and night. Yellow taxis weave through a labyrinth of bustling streets,
          their horns adding to the symphony of city life. Street vendors hawk their wares,
          their voices rising above the chatter of hurried footsteps. Amidst the towering skyscrapers,
          pockets of green offer a respite, where weary souls seek a moment of tranquility. The city never sleeps,
          a melting pot of dreams and ambitions, constantly evolving and reaching for the future."""

In [20]:
summary = summarizer(text, max_length = 50)

Your min_length=56 must be inferior than your max_length=50.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1570: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [21]:
print(summary)

[{'summary_text': 'In the heart of a sprawling metropolis, where steel and glass pierce the sky, a vibrant energy pulses day and night. Yellow taxis weave through a labyrinth of bustling streets, their horns adding to the symphony of city life.'}]


In [22]:
# Extract exact answer
answer = summary[0]['summary_text']
print(answer)

In the heart of a sprawling metropolis, where steel and glass pierce the sky, a vibrant energy pulses day and night. Yellow taxis weave through a labyrinth of bustling streets, their horns adding to the symphony of city life.


In [23]:
# check on user input
user_text = input("Enter text to summarize: ")
custom_max_length = int(input("Enter max length: "))
custom_summary = summarizer(user_text, max_length = custom_max_length)

Enter text to summarize: In the heart of a sprawling metropolis, where steel and glass pierce the sky, a vibrant energy pulses day and night. Yellow taxis weave through a labyrinth of bustling streets, their horns adding to the symphony of city life. Street vendors hawk their wares  their voices rising above the chatter of hurried footsteps. Amidst the towering skyscrapers, pockets of green offer a respite, where weary souls seek a moment of tranquility. The city never sleeps, a melting pot of dreams and ambitions, constantly evolving and reaching for the future.
Enter max length: 20


Your min_length=56 must be inferior than your max_length=20.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1570: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [24]:
print(custom_summary[0]['summary_text'])

In the heart of a sprawling metropolis, a vibrant energy pulses day and night.


In [25]:
# Saving model for later use

# Extract model
model = summarizer.model
# Extract tokenizer
tokenizer = summarizer.tokenizer

# Save both to a directory
save_directory = "saved_summarizer_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('saved_summarizer_model/tokenizer_config.json',
 'saved_summarizer_model/special_tokens_map.json',
 'saved_summarizer_model/vocab.json',
 'saved_summarizer_model/merges.txt',
 'saved_summarizer_model/added_tokens.json',
 'saved_summarizer_model/tokenizer.json')

# Gradio UI

In [26]:
!pip install gradio

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr

In [28]:
# Load the saved model and tokenizer
model_path = "saved_summarizer_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [29]:
# recreate pipeline
summarizer = pipeline(
    task = "summarization",
    model = model,
    tokenizer = tokenizer,
    device = 0
)

Device set to use cuda:0


In [30]:
# Define the Gradio interface function
def summarize_text(text, max_length=100):
  summary = summarizer(text, max_length = max_length, min_length=25, do_sample = False)
  return summary[0]['summary_text']

# Gradio UI
demo = gr.Interface(
    fn = summarize_text,
    inputs = [
        gr.Textbox(label="Enter Text to Summarize", lines=10, placeholder="Paste a long paragraph here..."),
        gr.Slider(30, 200, value=100, step=10, label="Max Summary Length")
    ],
    outputs = gr.Textbox(label="Summary"),
    title = "Text Summarization",
    description = "Enter a long paragraph and get a concise summary."
)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edb8d960ae984f9463.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
